In [1]:
#https://www.tensorflow.org/tutorials/quickstart/advanced 

import tensorflow as tf

print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Model

import tensorflow.keras as keras

import numpy as np

import matplotlib.pyplot as plt

#import tensorflow.keras.backend as K

2024-08-07 22:11:37.546874: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.10.0


In [2]:
# Universal parameters
batchSize = 1100 # batchSize = number of paths to be generated

d = 2 # dimension of forward process

batchSize_DNN = 1 #2 #10 # 20 50
Max_EPOCHS = 300
Mid_EPOCHS = 200
Min_EPOCHS = 100#60
lr = .001
seed = 0 #168
tf.random.set_seed(
    seed
)
tf.keras.utils.set_random_seed(
    seed
)

print('Number of paths (sample data) = ', batchSize)
print('batchSize = ', batchSize_DNN)
print('NumEpochs = ', Max_EPOCHS, Mid_EPOCHS, Min_EPOCHS)
print('Learning Rate = ', lr)



X_0 = 100 # initial price
print('initial price = ', X_0) 
V_0 = .15  # initial volatility
print('initial Vol = ', V_0)
K = 100


mu = .08
print('mu = ', mu )
alpha = 5 
print('alpha = ', alpha)
m = 0 
print('m = ', m)
nu = 1
print('nu = ', nu)
a = nu * np.sqrt(2 * alpha)
print('nu * sqrt(2 alpha ) = ', a)
rho = -.2
print('rho = ', rho)
gamma = 1
print('gamma = ', gamma )
eta = .2 
print('eta = ', eta)
timeSteps = 10  #20 #50 
print('Total number of time steps = ', timeSteps )
r = 0.02 # .05
print('risk free rate r = ', r)
T = 1/4 # terminal time
print('T = ', T)
dt = T / timeSteps
print('dt = ', dt)

strikes = [70,75,80,85,90,95,100,105,110,115]
# strikes = [i for i in range(70,116,2)]
#print('strikes = ', strikes)
#print('logMoney = ', np.array(strikes) / X_0)
# for loop here 
#ivValues = []
print('-------------Beginning computation-----------------')




Number of paths (sample data) =  1100
batchSize =  1
NumEpochs =  300 200 100
Learning Rate =  0.001
initial price =  100
initial Vol =  0.15
mu =  0.08
alpha =  5
m =  0
nu =  1
nu * sqrt(2 alpha ) =  3.1622776601683795
rho =  -0.2
gamma =  1
eta =  0.2
Total number of time steps =  10
risk free rate r =  0.02
T =  0.25
dt =  0.025
-------------Beginning computation-----------------


In [3]:
def trainingData(seed): # = 168):
    tf.random.set_seed(seed)
    std_normal = tf.random.normal([batchSize,  # [x,   [number of paths,
                                   timeSteps + 1,  #  y, =  number of values in each path,            = tensor shape
                                   2],         #  z]    dim of each path 
                                  mean = 0.0,
                                  stddev = 1.0,
                                  dtype = tf.dtypes.float32,
                                  seed = seed,
                                  name = None)
    dW = std_normal * np.sqrt(dt)
    X = tf.Variable(tf.ones([batchSize, timeSteps + 1, 2], 
                            dtype = tf.dtypes.float32) * tf.constant([X_0, V_0]))
    for i in range(timeSteps):
        stockTerm1 = mu * X[:,i, 0] * dt
        stockTerm2 = (.7 / (np.pi) * (tf.math.atan(X[:,i, 1] - 1) + np.pi/2) + .03) * X[:, i,0] * dW[:,i,0] # sigma(Y_t)
        stockTerm3 = -r * X[:,i,0] * dt
        X[:, i+1, 0].assign(X[:,i, 0] + stockTerm1 + stockTerm2 + stockTerm3)
        volTerm1 = alpha * (m - X[:,i,1]) * dt
        volTerm2 = a * (rho *dW[:,i,0] + np.sqrt(1 - rho ** 2)* dW[:,i,1])
        X[:, i+1, 1].assign(X[:, i, 1] + volTerm1 + volTerm2) 
        
       # X = X[:, i+1, :].assign(X[:, i, :] + r*X[:, i, :]*dt + sigma*X[:, i, :]*dW[:, i, :])
    return X, dW

    

#terminal condition
def fun_g(X, K, t):
    # print(X.shape)
    g = tf.math.maximum(K * tf.math.exp((-r * (t/timeSteps) * T)) - X, 0)
    return g







def fun_bdy(z,y):
    F = z + y
    return F

In [4]:
X, dW = trainingData(seed)
print('sample path = ', X[1,:,:])
# print('sample brownian motion = ', dW[0,:,:])
print('sample BM mean: ', tf.math.reduce_mean(dW), '| variance: ', tf.math.reduce_variance(dW))
                    

sample path =  tf.Tensor(
[[ 1.00000000e+02  1.50000006e-01]
 [ 1.00554573e+02 -2.82164752e-01]
 [ 1.02307930e+02 -1.57119930e-02]
 [ 9.80057449e+01 -2.48888656e-01]
 [ 1.04667213e+02 -3.32593203e-01]
 [ 1.06612518e+02 -3.46326292e-01]
 [ 1.06523796e+02 -4.39549088e-02]
 [ 1.08641037e+02 -5.97284615e-01]
 [ 1.10312263e+02  2.68377781e-01]
 [ 1.19024925e+02  5.41483462e-01]
 [ 1.06819839e+02  5.83199024e-01]], shape=(11, 2), dtype=float32)
sample BM mean:  tf.Tensor(0.001961416, shape=(), dtype=float32) | variance:  tf.Tensor(0.024696153, shape=(), dtype=float32)


2024-08-07 22:11:42.732228: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
X, dW = trainingData(seed)

In [6]:
print('Computing the Boundary')
#X, dW = trainingData(seed)
for K in strikes:
    print('Strike = ', K)
    L = fun_g(X[:, timeSteps, 0], K, timeSteps)
    #print(L.shape)
    print(tf.math.reduce_mean(X[:, timeSteps, 0]))
    print(tf.math.reduce_mean(L))
    # dtype = tf.dtypes.float32))#torch.empty([numEpochs, batchSize, timeSteps])
    np.savetxt("Boundarypredictions-Seller_"+str(timeSteps)+"K_"+str(K)+".csv", L, delimiter = ",")
    
    for time in range(timeSteps -1, -1, -1):
        #print('timeStep =', time)
        z = np.loadtxt("BSDE0predictions_"+str(time)+"_y.csv", delimiter = ",", dtype = 'float32')
        print(z.shape)
        #z = z.reshape((1000, 1))
        y = fun_g(X[:, time, 0], K, time)
        #print(y.shape)

        L = fun_bdy(z,y) 
        print('timestep = ', time, 'Boundary = ', tf.math.reduce_mean(L))
        np.savetxt("Boundarypredictions-Seller_"+str(time)+"K_"+str(K)+".csv", L, delimiter = ",")    

print('Boundary Computed')
print('============Computing the RBSDE============')
    
    
    
    


Computing the Boundary
Strike =  70
tf.Tensor(102.154106, shape=(), dtype=float32)
tf.Tensor(0.10117897, shape=(), dtype=float32)
(1100,)
timestep =  9 Boundary =  tf.Tensor(0.081680715, shape=(), dtype=float32)
(1100,)
timestep =  8 Boundary =  tf.Tensor(0.031859472, shape=(), dtype=float32)
(1100,)
timestep =  7 Boundary =  tf.Tensor(0.019570295, shape=(), dtype=float32)
(1100,)
timestep =  6 Boundary =  tf.Tensor(0.005231535, shape=(), dtype=float32)
(1100,)
timestep =  5 Boundary =  tf.Tensor(-0.0038321863, shape=(), dtype=float32)
(1100,)
timestep =  4 Boundary =  tf.Tensor(-0.008656919, shape=(), dtype=float32)
(1100,)
timestep =  3 Boundary =  tf.Tensor(-0.009117508, shape=(), dtype=float32)
(1100,)
timestep =  2 Boundary =  tf.Tensor(-0.009653919, shape=(), dtype=float32)
(1100,)
timestep =  1 Boundary =  tf.Tensor(-0.010844309, shape=(), dtype=float32)
(1100,)
timestep =  0 Boundary =  tf.Tensor(-0.0117490105, shape=(), dtype=float32)
Strike =  75
tf.Tensor(102.154106, shape=(